In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical

In [2]:
# 下载Keras提供的手写数字图像数据。
np.random.seed(0)
(X_train_base, labels_train_base), (X_test, labels_test) = mnist.load_data()

# 将训练集分成训练数据（X_train, labels_train）和验证数据（X_validation, labels_validation），比例为8：2。
X_train,X_validation,labels_train,labels_validation = train_test_split(X_train_base,labels_train_base,test_size = 0.2)

In [3]:
# 每个图像都是一个矩阵，所以要转换为1D→覆盖X_train, X_validation, X_test
X_train = X_train.reshape(-1,784)
X_validation = X_validation.reshape(-1,784)
X_test = X_test.reshape(-1,784)

#正常化
X_train = X_train.astype('float32')
X_validation = X_validation.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_validation /= 255
X_test /= 255

In [4]:
# labels_train、labels_validation和labels_test被制成虚拟变量并存储在y_train、y_validation和y_test中。
y_train = to_categorical(labels_train)
y_validation = to_categorical(labels_validation)
y_test = to_categorical(labels_test)

In [5]:
# 参数设置
n_features = 784
n_hidden   = 100
bias_init = 0.1

# 学习率
rate       = 0.01

# 使用Sequential类准备好模型。
model = Sequential()

# 额外的隐藏层
model.add(Dense(n_hidden,activation='relu',input_shape=(n_features,)))
model.add(Dense(n_hidden,activation='relu'))
model.add(Dense(n_hidden,activation='relu'))

# 附加输出层
model.add(Dense(10,activation='softmax'))

In [11]:
# 构建TensorFlow模型
model.compile(optimizer=tf.optimizers.Adam(rate),
              loss='categorical_crossentropy',
              metrics=['mae', 'accuracy'])

# 适用于早期停止和装配
log = model.fit(X_train, y_train, epochs=3000, 
                batch_size=100, verbose=True,
                callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                         min_delta=0, patience=10, verbose=1)],
                validation_data=(X_validation, y_validation))

Epoch 1/3000
480/480 [==============================] - 4s 6ms/step - loss: 0.0637 - mae: 0.0031 - accuracy: 0.9890 - val_loss: 0.2526 - val_mae: 0.0066 - val_accuracy: 0.9704
Epoch 2/3000
480/480 [==============================] - 2s 5ms/step - loss: 0.0485 - mae: 0.0027 - accuracy: 0.9905 - val_loss: 0.2579 - val_mae: 0.0073 - val_accuracy: 0.9711
Epoch 3/3000
480/480 [==============================] - 2s 5ms/step - loss: 0.0653 - mae: 0.0033 - accuracy: 0.9879 - val_loss: 0.2669 - val_mae: 0.0076 - val_accuracy: 0.9703
Epoch 4/3000
480/480 [==============================] - 3s 5ms/step - loss: 0.0631 - mae: 0.0032 - accuracy: 0.9881 - val_loss: 0.2527 - val_mae: 0.0066 - val_accuracy: 0.9705
Epoch 5/3000
480/480 [==============================] - 2s 5ms/step - loss: 0.0543 - mae: 0.0031 - accuracy: 0.9886 - val_loss: 0.3202 - val_mae: 0.0088 - val_accuracy: 0.9626
Epoch 6/3000
480/480 [==============================] - 2s 5ms/step - loss: 0.0525 - mae: 0.0031 - accuracy: 0.9893 - va

In [21]:
# 用测试数据运行预测。
pred_test = model.predict(X_test)
pred_tesr=np.argmax(pred_test,axis=1)

validation = (pred_tesr == labels_test)
size=validation.size
correct = np.count_nonzero(validation)
print(f"{correct}/{size} correct ({correct*100/size:.3f}%)")

313/313 [==============================] - 1s 2ms/step
9705/10000 correct (97.050%)
